# Seq2Seq

*Sequence to Sequence*.  
*Encoder-Decoder Model*とも。

2つのモデルを用いて、入力されたシーケンスに基づいた別のシーケンスを出力するモデル。

In [1]:
# from xml.etree import ElementTree as ET
# from glob import glob
# import os
import os; os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import warnings; warnings.filterwarnings('ignore')
from typing import List
import random

import tensorflow as tf
import tensorflow_datasets as tfds
import sentencepiece as spm
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence
from dlprog import train_progress

In [2]:
prog = train_progress(with_test=True)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

### 発想

これまで、RNNを用いて、入力した単語列に続く単語を予測するモデルを作成し、単語の予測を繰り返すことで文章を生成した。

RNNは時系列の情報を保持するために隠れ状態$h_t$を用いる。隠れ状態の初期値$h_0$は0ベクトルとしているが、ここで、何らかの入力データから生成したベクトルを$h_0$として用いることを考える。このとき、上手く学習させれば、その入力に基づいた文章を生成できそう。

例えば、入力を画像をとし、CNNを用いて抽出した特徴量を$h_0$として用いるようにすれば、入力画像に基づいた文章が生成できる。画像のキャプションなどが例に挙げられる。  
学習方法は簡単で、入力画像に対して適切な文章が出力されるように学習させるだけ。隠れ状態を通じてRNNからCNNまで逆伝播を繋げる。

では、入力に文章を用いることはできないだろうか。RNNに文章を入力し、最後に出力された隠れ状態を文章ベクトルとする。これを別のRNNへの入力$h_0$とすれば、入力文に基づいた文章生成が可能になる。  

この発想は翻訳タスクに大きく役立つ。入力と出力に同じ意味を持った異なる言語の文章を設定すれば、入力文と同じ意味を持った文章生成が可能になる。  
こういった、シーケンスをシーケンスに変換するモデルを *seq2seq (Sequence to Sequence)* と呼ぶ。

本章では、入力に日本語文、出力に入力と同じ意味を持つ英語文を設定し、日本語→英語の翻訳を行うモデルを作成する。

このモデルは以下の二つのRNNから構成される。
- ***Encoder*** : 文章ベクトルを生成するRNN
- ***Decoder*** : 文章ベクトルを受け取って出力文を生成するRNN

このことから***Encoder-Decoderモデル***とも呼ばれる。


---

## 対訳コーパス

翻訳モデルを作るには、同じ意味を持つ文章が複数の言語でまとまっているデータが必要。このようなデータは対訳コーパスと呼んだりする。

本章では以下のデータセットを使用する。
- [iwslt2017  |  TensorFlow Datasets](https://www.tensorflow.org/datasets/community_catalog/huggingface/iwslt2017?hl=ja#iwslt2017-en-ja)

In [3]:
ds = tfds.load(
    'huggingface:iwslt2017/iwslt2017-en-ja',
    data_dir='data',
    split='train'
)
ds = list(ds.as_numpy_iterator())

In [4]:
data_ja = []
data_en = []
for sample in ds:
    ja = sample['translation']['ja'].decode()
    en = sample['translation']['en'].decode()
    data_ja.append(ja)
    data_en.append(en)

print('num of data:', len(data_ja), '\n')
for _ in range(5):
    i = random.randint(0, len(data_ja))
    print(data_ja[i])
    print(data_en[i])
    print()

num of data: 223108 

人々が衣食住を持たずに 繁栄の話しをするのはナンセンスです
It would be nonsense to talk about people flourishing if they didn't have food, clothing and shelter.

屋根を突き抜けて立つ ３本の木があるのです
There are three trees popping through.

最初に現れるものに手を染める
So they take the first one that comes along, often.

もちろんオーヴィルとウィルバーのライト兄弟、 ライトフライヤー号です
So this is of course Orville and Wilbur Wright, and the Wright Flyer.

次の犬は 亡霊となって語りかけてきます 犬の魂が再び主人に会いに この世に戻ってきたのです
And our next dog speaks in something called the revenant, which means a spirit that comes back to visit you.



書き出し

In [5]:
textfile_ja = 'data/iwslt2017_ja.txt'
with open(textfile_ja, 'w') as f:
    f.write('\n'.join(data_ja))

textfile_en = 'data/iwslt2017_en.txt'
with open(textfile_en, 'w') as f:
    f.write('\n'.join(data_en))

データ数を減らしたものも作っておく。

In [6]:
n_data = 10000
data_ja = data_ja[:n_data]
data_en = data_en[:n_data]

textfile_ja = f'data/kyoto_ja_{n_data}.txt'
with open(textfile_ja, 'w') as f:
    f.write('\n'.join(data_ja))

textfile_en = f'data/kyoto_en_{n_data}.txt'
with open(textfile_en, 'w') as f:
    f.write('\n'.join(data_en))

トークナイザの学習。

In [7]:
pad_id = 3

vocab_size_ja = 8000
tokenizer_prefix_ja = f'models/tokenizer_iwslt2017_ja_{n_data}'
spm.SentencePieceTrainer.Train(
    input=textfile_ja, # データセット
    model_prefix=tokenizer_prefix_ja,
    vocab_size=vocab_size_ja,
    pad_id=pad_id
)

vocab_size_en = 8000
tokenizer_prefix_en = f'models/tokenizer_iwslt2017_en_{n_data}'
spm.SentencePieceTrainer.Train(
    input=textfile_en, # データセット
    model_prefix=tokenizer_prefix_en,
    vocab_size=vocab_size_en,
    pad_id=pad_id
)

sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: data/kyoto_ja_10000.txt
  input_format: 
  model_prefix: models/tokenizer_iwslt2017_ja_10000
  model_type: UNIGRAM
  vocab_size: 8000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: 3
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
  enable_differential_privacy: 0

### 前処理

データ数を減らした方を使う。

トークナイザ読み込み

In [8]:
sp_ja = spm.SentencePieceProcessor(f'{tokenizer_prefix_ja}.model')
sp_en = spm.SentencePieceProcessor(f'{tokenizer_prefix_en}.model')

unk_id = sp_ja.unk_id()
bos_id = sp_ja.bos_id()
eos_id = sp_ja.eos_id()

n_vocab_ja = len(sp_ja)
n_vocab_en = len(sp_en)
print('num of vocabrary (ja):', n_vocab_ja)
print('num of vocabrary (en):', n_vocab_en)

num of vocabrary (ja): 8000
num of vocabrary (en): 8000


トークン化

In [9]:
data_ids_ja = sp_ja.encode(data_ja)
data_ids_en = sp_en.encode(data_en)

BOS, EOSの追加

In [10]:
for ids_ja, ids_en in zip(data_ids_ja, data_ids_en):
    ids_en.insert(0, bos_id)
    ids_ja.append(eos_id)
    ids_en.append(eos_id)

### 学習データの作成

入力文と正解のペアを作成する。

Encoderへの入力（入力文）とDecoderの出力（正解）のペアを作成する。  
また、Decoderへの入力を考える必要がある。今回は教師強制を採用し、出力文の頭に\<BOS>を付与したものをDecoderへの入力とする。

例）
Encoderへの入力（入力文） | Decoderへの入力 | Eecoderの出力（出力文）
--- | --- | ---
夏 休み が 終わり ました 。 \<EOS> | \<BOS> Summer vacation is over . | Summer vacation is over . \<EOS>
ツイッター は 亡くなり ました 。 \<EOS> | \<BOS> Twitter is dead . | Twitter is dead . \<EOS>
今日 から X で 暮らし ましょう 。 \<EOS> | \<BOS> Let 's live in X from today . | Let 's live in X from today . \<EOS>

DataLoaderの作成。

In [11]:
class TextDataset(Dataset):
    def __init__(self, data_ids_ja, data_ids_en):
        self.data_ja = [torch.tensor(ids) for ids in data_ids_ja]
        self.data_en = [torch.tensor(ids) for ids in data_ids_en]
        self.n_data = len(self.data_ja)

    def __getitem__(self, idx):
        ja = self.data_ja[idx]
        en = self.data_en[idx]
        x_enc = ja # encoderへの入力
        x_dec = en[:-1] # decoderへの入力
        y_dec = en[1:] # decoderの出力
        return x_enc, x_dec, y_dec

    def __len__(self):
        return self.n_data

def collate_fn(batch): # padding
    x_enc, x_dec, y_dec= zip(*batch)
    x_enc = pad_sequence(x_enc, batch_first=True, padding_value=pad_id)
    x_dec = pad_sequence(x_dec, batch_first=True, padding_value=pad_id)
    y_dec = pad_sequence(y_dec, batch_first=True, padding_value=pad_id)
    return x_enc, x_dec, y_dec

dataset = TextDataset(data_ids_ja, data_ids_en)
train_dataset, test_dataset = random_split(dataset, [0.8, 0.2])

batch_size = 32
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True,
    collate_fn=collate_fn
)
test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    drop_last=True,
    collate_fn=collate_fn
)

x_enc, x_dec, y_dec = next(iter(train_loader))
x_enc.shape, x_dec.shape, y_dec.shape

(torch.Size([32, 72]), torch.Size([32, 55]), torch.Size([32, 55]))


---

## モデル構築

EncoderとDecoderを作る。

### Encoder

入力文を入れて隠れ状態を出力するだけのRNN(LSTM)。LSTMと線形層で作る。

ここで1つ工夫を加える。普通に作っても動きはするが、学習が上手くいかない可能性がある。

encoderへの入力はpaddingされたデータである。ここで、paddingされた範囲が多い=padトークンが多く含まれているデータは、padの数が多くなるにつれて、隠れ状態がある一定の値に収束してしまう。RNNに同じトークンを何度も入力することで隠れ状態が収束していってしまうのだ。  
そうなってしまうと、入力文に依る隠れ状態の違いが少なくなり、翻訳を学習できない。

そこで、encoderが出力する隠れ状態として、padトークンを除いた最後のトークン=EOSを入力した時点のものを使用する。  

以上を踏まえ、encoderを以下のように実装する。

In [12]:
class Encoder(nn.Module):
    def __init__(self, n_vocab, embed_size, hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(n_vocab, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, hidden_size)

    def forward(self, x):
        """
        x: (batch_size, seq_len)
        """
        eos_positions = x == eos_id
            # eosに対応する位置のみがTrueとなったTensor: (batch_size, seq_len)
        x = self.embedding(x) # (batch_size, seq_len, embed_size)
        hs, _ = self.lstm(x) # (batch_size, seq_len, hidden_size)
        h = hs[eos_positions] # (batch_size, hidden_size)
        h = self.fc(h).unsqueeze(0) # (1, batch_size, hidden_size)
        return h

### Decoder

Encoderから出力された隠れ状態を受け取り、出力文を生成するRNN。Encoder同様、LSTMと線形層で作る。

In [13]:
class Decoder(nn.Module):
    def __init__(self, n_vocab, embed_size, hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(n_vocab, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, n_vocab)

    def forward(self, x, hc):
        x = self.embedding(x) # (batch_size, seq_len, embed_size)
        hs, (h, c) = self.lstm(x, hc)
            # hs: (batch_size, seq_len, hidden_size)
            # h: (1, batch_size, hidden_size)
            # c: (1, batch_size, hidden_size)
        y = self.fc(hs) # (batch_size, seq_len, n_vocab)
        return y, (h, c)

### Seq2Seq

EncoderとDecoderを合わせて、入力から出力までの一連の処理を行うモデルを作る。

In [14]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, x_enc, x_dec):
        h = self.encoder(x_enc)
        hc = (h, torch.zeros_like(h))
        y, _ = self.decoder(x_dec, hc)
        return y

### 学習

In [15]:
cross_entropy = nn.CrossEntropyLoss(ignore_index=pad_id)
def loss_fn(y, t):
    loss = cross_entropy(y.reshape(-1, n_vocab_ja), t.ravel())
    return loss

def eval_model(model):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for x_enc, x_dec, y_dec in test_loader:
            x_enc = x_enc.to(device)
            x_dec = x_dec.to(device)
            y_dec = y_dec.to(device)
            y = model(x_enc, x_dec)
            loss = loss_fn(y, y_dec)
            total_loss += loss.item()
    loss = total_loss / len(test_loader)
    return loss

def train(model, optimizer, n_epochs, prog_unit=1):
    prog.start(n_iter=len(train_loader), n_epochs=n_epochs, unit=prog_unit)
    for _ in range(n_epochs):
        model.train()
        for x_enc, x_dec, y_dec in train_loader:
            optimizer.zero_grad()
            x_enc = x_enc.to(device)
            x_dec = x_dec.to(device)
            y_dec = y_dec.to(device)

            y = model(x_enc, x_dec)
            loss = loss_fn(y, y_dec)
            loss.backward()
            optimizer.step()
            prog.update(loss.item())

        if prog.now_epoch % prog_unit == 0:
            test_loss = eval_model(model)
            prog.memo(f'test: {test_loss:.5f}', no_step=True)
        prog.memo()

In [16]:
hidden_size, embed_size = 512, 512
encoder = Encoder(n_vocab_ja, embed_size, hidden_size)
decoder = Decoder(n_vocab_en, embed_size, hidden_size)
model = Seq2Seq(encoder, decoder).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [17]:
train(model, optimizer, n_epochs=200, prog_unit=20)

   1-20/200: ############################## 100% [00:02:51.26] loss train: 4.88085, test: 4.91392 
  21-40/200: ############################## 100% [00:02:09.91] loss train: 3.56565, test: 5.17883 
  41-60/200: ############################## 100% [00:02:09.57] loss train: 2.55636, test: 5.72575 
  61-80/200: ############################## 100% [00:02:06.33] loss train: 1.74147, test: 6.36254 
 81-100/200: ############################## 100% [00:02:05.70] loss train: 1.12456, test: 7.03689 
101-120/200: ############################## 100% [00:02:05.42] loss train: 0.68134, test: 7.78627 
121-140/200: ############################## 100% [00:02:05.53] loss train: 0.38464, test: 8.41202 
141-160/200: ############################## 100% [00:02:05.25] loss train: 0.20159, test: 8.95770 
161-180/200: ############################## 100% [00:02:05.92] loss train: 0.10116, test: 9.36043 
181-200/200: ############################## 100% [00:02:05.48] loss train: 0.05881, test: 9.83690 



---

## 翻訳

作成したモデルに日本語文を入力し、英語に翻訳して出力する。

In [20]:
def token_sampling(y, decisive=True):
    y.squeeze_(0)
    if decisive:
            token = y.argmax().item()
    else:
        y[unk_id] = -torch.inf
        probs = F.softmax(y, dim=-1)
        token, = random.choices(range(n_vocab_en), weights=probs)
    return token


bos_id = sp_en.bos_id()
eos_id = sp_en.eos_id()
@torch.no_grad()
def translate(
    model: nn.Module,
    in_text: str, # 入力文（日本語）
    max_len: int = 100, # 出力のトークン数の上限
    decisive: bool = True, # サンプリングを決定的にするか
) -> str:
    model.eval()
    in_ids = sp_ja.encode(in_text)
    in_ids = torch.tensor(in_ids + [eos_id], device=device)

    h = model.encoder(in_ids)
    hc = (h, torch.zeros_like(h))
    next_token = bos_id

    token_ids = []
    while len(token_ids) <= max_len and next_token != eos_id:
        x = torch.tensor([[next_token]], device=device)
        y, hc = model.decoder(x, hc)
        next_token = token_sampling(y, decisive)
        token_ids.append(next_token)

    sentence = sp_en.decode(token_ids)
    return sentence

まずは訓練データから。

In [33]:
n = 5
for _ in range(n):
    i = random.randint(0, len(train_dataset))
    x, _, t = train_dataset[i]
    x = sp_ja.decode(x.tolist())
    t = sp_en.decode(t.tolist())
    print('input:', x)
    print('output:', translate(model, x))
    print('answer:', t)
    print()

input: これが重要なのです
output: And that makes a difference.
answer: And that makes a difference.

input: 長期的に考えると 生命がいなければ自分たちで作ることになります
output: But in the long run, if there's no life there, we create it ourselves.
answer: But in the long run, if there's no life there, we create it ourselves.

input: 実は要となるピッチはすでに登り終えています
output: And all the hard pitches are actually below him.
answer: And all the hard pitches are actually below him.

input: 実はここは生命と無縁ではありません ここは私たちが発見した中で最も
output: Because contrary to what you might think, this is not devoid of life.
answer: Because contrary to what you might think, this is not devoid of life.

input: 新世界ザルとも呼ばれるのは 約3500万年前に
output: These guys are New World primates, about 35 million years ago.
answer: These guys are New World primates, about 35 million years ago.



一致。

訓練データに含まれていないものも試してみる。

In [35]:
# test data
for _ in range(n):
    i = random.randint(0, len(test_dataset))
    x, _, t = test_dataset[i]
    x = sp_ja.decode(x.tolist())
    t = sp_en.decode(t.tolist())
    print('input:', x)
    print('output:', translate(model, x))
    print('answer:', t)
    print()

input: それは現実世界へも拡がるだろうと思います
output: Here's the poor: you know, the starts and say, "I love you.
answer: I even think it can go into the real world.

input: だから 科学と技術によって 効率が高く 太陽エネルギーを活用し 知的生産に基づく経済へと スムーズに移行できて 2050年には 90億人が豊かに暮らし デジタル生活を送れるという 考えは妄想です
output: So if the whole project will be remembered up, then thinking about it  ⁇  And then, then I feel like the first day that we got to understand from what was feeling of a living human up, so he couldn't make any moneying for life.
answer: So the idea that we can smoothly transition to a highly-efficient, solar-powered, knowledge-based economy transformed by science and technology so that nine billion people can live in 2050 a life of abundance and digital downloads is a delusion.

input: 私たちが気付いた2つ目の問題は 極めて不十分な 退役軍人への社会復帰支援です イラクやアフガンから 退役軍人が復員するに従い 新聞に大きく取り上げられています 彼らは市民生活に復帰することに 大変苦戦しているのです
output: In 1947, there was no wind tunnel data And yet, on Tuesday, October 14th, 1947, Chuck Yeager climbed into the cockpit of his Bell  ⁇

In [23]:
# original
sentences = [
    '私は猫です。',
    '上手く文章が書けるようになりました。'
]

for sentence in sentences:
    print('input:', sentence)
    print('output:', translate(model, sentence))
    print()

input: 私は猫です。
output: So I'm finish.

input: 上手く文章が書けるようになりました。
output: Well, you know, I think better as the best way around the life.



びみょう。